## QR-разложение матрицы

Вариант 17

In [48]:
import numpy as np

N = 3
A = np.array([-6, 1, -4, -6, 8, -2, 2, -9, 5])
A = np.reshape(A, (N, N))
A

array([[-6,  1, -4],
       [-6,  8, -2],
       [ 2, -9,  5]])

Используя вектор $\mathbf{x}$, и гиперплоскость с вектором нормали $\mathbf{u}$, преобразование Хаусхолдера отражает $\mathbf{x}$ относительно гиперплоскости.

Матрица этого преобразования:

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Для двух векторов одинаковой длины $\mathbf{x}$ and $\mathbf{y}$, поворот, переводящий $\mathbf{x}$ в $\mathbf{y}$ - это преобразование Хаусхолдера со следующим вектором нормали:

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Следующая функция поворачивает вектор $\mathbf{x} = (x_1, \dots, x_n)$ в вектор $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ с использованием преобразования Хаусхолдера

In [49]:
def householder(vec): 
    #функция считает преобразование Хаусхолдера для заданного вектора и возвращает его матрицу H и итоговый вектор
    
    vec = np.asarray(vec, dtype = float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    m = vec.shape[0]
    vec = vec.reshape((m, 1))
    if m == 1:
        e = np.asarray([1])
        e = e.reshape((e.shape[0], 1))
        return np.asarray(vec), e
    xlen = length(vec)
    y = np.zeros(vec.shape, dtype = float)
    y[0] = xlen
    
    numerator = 0 # Так лучше считать разность между векторами чтобы избежать ошибки при округлении
    for xi in vec[1:]:
        numerator -= (xi**2)
    denominator = vec[0] + xlen
    diff_x1_y = numerator/denominator
    diff_rest = vec[1:]
    diff = np.insert(diff_rest, 0, diff_x1_y)
    difflen = length(diff)
    
    u = np.divide(diff, difflen)
    u = u.reshape(m, 1)
    ut = np.transpose(u)
    one = np.eye(m, dtype = float)
    H = one - np.dot(2,(np.dot(u, ut)))
    #print(H)
    Hx = np.dot(H, vec)
    #print(Hx)
    return Hx, H # можно было бы вернуть и y, которому равен Hx
    
def length(vec): #2-норма вектора
    xlen = 0 
    for xi in vec:
        xlen += (xi**2)
    xlen = np.sqrt(xlen)
    return xlen

Для матрицы $\mathbf{A}$ размерами $m\times n$, мы строим преобразование Хаусхолдера $\mathbf{H}_1$, которое преобразует первый столбец  $\mathbf{A}$ (и называем получившуюся матрицу $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

То же самое мы делаем с подматрицей (правой нижней) матрицы $\mathbf{A}^{(1)}$, следующее преобразование зануляет поддиагональные элементы во втором столбце матрицы $\mathbf{A}$:

Повторив этот процесс $n-1$ раз, мы получим

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

где $\mathbf{R}$ - верхняя треугольняя матрица, а $\mathbf{Q}$ - ортогональная (как обратная к произведению ортогональных). Мы получили $\mathrm{QR}$ разложение матрицы $\mathbf{A}$. 

In [50]:
def qr_decomp(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    dim = max(n, m)
    Q = np.eye(dim)
    for i in range(n):
        col = a1[i:, i]
        Hcol, H = householder(col)
        matr = np.eye(dim) #заполняем пустые места единичными подматрицами
        matr[i:, i:] = H[:,:] #это нужно так как каждая следующая матрица H меньше предыдущей
        a1 = matr @ a1 
        Q = Q @ matr
    return Q, a1

In [51]:
Q, R = qr_decomp(A)
np.set_printoptions(suppress=True) #убираем всякие 1e-15 в ноль для красоты
print("QR разложение матрицы A: \nQ:\n" + str(Q) + "\nR:\n" + str(R))

QR разложение матрицы A: 
Q:
[[-0.6882472  -0.53109962  0.49421552]
 [-0.6882472   0.2625661  -0.67629493]
 [ 0.22941573 -0.80560054 -0.54623821]]
R:
[[ 8.71779789 -8.25896642  5.27656188]
 [ 0.          8.81983411 -2.42873646]
 [-0.         -0.         -3.3554633 ]]


QR разложение мы используем для построения QR-алгоритма вычисления собственных значений матрицы. Строится следующий итерационный процесс: 

$A^{(0)} = A$

$A^{(0)} = Q^{(0)}R^{(0)}$

$A^{(1)} = R^{(1)}Q^{(1)}$

...

$A^{(k)} = Q^{(k)}R^{(k)}$

$A^{(k+1)} = R^{(k)}Q^{(k)}$

Последовательность, полученная таким образом, будет сходиться к верхней треугольной матрице если нет комплексно-сопряжённых пар собственных значений, и к квазитреугольной, если они есть.

Каждому вещественному собственному значению будет соответствовать столбец со стремящимися к нулю поддиагональными элементами, каждой паре комплексно-сопряжённых - блок 2х2, для которого начиная с некоторой итерации корни уравнения $(a_{jj}^{(k)} - \lambda^{(k)})(a_{j+1j+1}^{(k)} - \lambda^{(k)}) = a_{jj+1}^{(k)}a_{j+1j}^{(k)}  $ отличаются незначительно

In [52]:
def eigenvalues(A, eps):
    maxiter = 1/eps
    N = A.shape[0]
    B = A.copy()
    evalues = np.full((N,), np.inf, dtype = complex)
    roots = []
    for i in range(N-1):
        roots.append([np.inf, np.inf])
    it = 0
    while(np.inf in evalues):
        B_prev = B
        Q, R = qr_decomp(B)
        B = R @ Q
        sum_col = 0
        roots_prev = roots.copy()
        for j in range(N-1):
            coefs = []
            coefs.append(1)
            coefs.append(B[j+1, j+1] - B[j, j])
            coefs.append(B[j, j]*B[j+1, j+1] - B[j, j+1]*B[j+1, j])
            roots[j] = np.roots(coefs)
        for j in range(N):
            for i in range(j+1, N):
                sum_col += B[i, j] ** 2
            sum_col = np.sqrt(sum_col)
            #print("Сумма колонки " +str(j+1) + ": " + str(sum_col))
            if sum_col <= eps:
                evalues[j] = B[j, j]
            elif(it > maxiter and j < N-1 and sum_col > eps):
                for i in range(N-1):
                    #print("R: " + str(roots[j][i]) + " R_prev: " + str(roots_prev[j][i]))
                    if np.absolute(roots[j][i] - roots_prev[j][i]) <= eps:
                        evalues[j] = roots[j][0]
                        evalues[j+1] = roots[j][1]
        it += 1
        #print(evalues)
    return evalues

In [53]:
evs = eigenvalues(A, 1e-7)
evs

array([ 8.32134993+0.j, -6.26790924+0.j,  4.94655931+0.j])

In [45]:
A = np.array([1,3,1,1,1,4,4,3,1])
A = np.reshape(A, (3,3))
evs = eigenvalues(A, 1e-2)
evs

R: (-0.23530751971054517+2.2687831538795225j) R_prev: (0.8300266820350806+2.1245710944258076j)
R: (-0.23530751971054517-2.2687831538795225j) R_prev: (0.8300266820350806-2.1245710944258076j)
R: (-0.7224198354042803+2.163528647735983j) R_prev: (-0.23530751971054517+2.2687831538795225j)
R: (-0.7224198354042803-2.163528647735983j) R_prev: (-0.23530751971054517-2.2687831538795225j)
R: (0.0072632572462248985+2.2809414445035663j) R_prev: (-0.7224198354042803+2.163528647735983j)
R: (0.0072632572462248985-2.2809414445035663j) R_prev: (-0.7224198354042803-2.163528647735983j)
R: (0.7337159617847296+2.1597239442998606j) R_prev: (0.0072632572462248985+2.2809414445035663j)
R: (0.7337159617847296-2.1597239442998606j) R_prev: (0.0072632572462248985-2.2809414445035663j)
R: (0.19491092996517892+2.272610031998439j) R_prev: (0.7337159617847296+2.1597239442998606j)
R: (0.19491092996517892-2.272610031998439j) R_prev: (0.7337159617847296-2.1597239442998606j)
R: (-0.8227831482041301+2.1273867817562313j) R_pre

R: (-0.3556314727809571+2.25305856198357j) R_prev: (0.8488428204659186+2.1171236369903603j)
R: (-0.3556314727809571-2.25305856198357j) R_prev: (0.8488428204659186-2.1171236369903603j)
R: (-0.6857772614470145+2.175420919234267j) R_prev: (-0.3556314727809571+2.25305856198357j)
R: (-0.6857772614470145-2.175420919234267j) R_prev: (-0.3556314727809571-2.25305856198357j)
R: (0.052345132462747315+2.28035230069112j) R_prev: (-0.6857772614470145+2.175420919234267j)
R: (0.052345132462747315-2.28035230069112j) R_prev: (-0.6857772614470145-2.175420919234267j)
R: (0.7668959117936662+2.1481660291127387j) R_prev: (0.052345132462747315+2.28035230069112j)
R: (0.7668959117936662-2.1481660291127387j) R_prev: (0.052345132462747315-2.28035230069112j)
R: (0.06627391752349487+2.2799899991043637j) R_prev: (0.7668959117936662+2.1481660291127387j)
R: (0.06627391752349487-2.2799899991043637j) R_prev: (0.7668959117936662-2.1481660291127387j)
R: (-0.7971346605955056+2.137129607917385j) R_prev: (0.06627391752349487

R: (-0.6471190838923697+2.187231930871102j) R_prev: (-0.46601990042177177+2.2328394659201787j)
R: (-0.6471190838923697-2.187231930871102j) R_prev: (-0.46601990042177177-2.2328394659201787j)
R: (0.09741929659485182+2.278871674493936j) R_prev: (-0.6471190838923697+2.187231930871102j)
R: (0.09741929659485182-2.278871674493936j) R_prev: (-0.6471190838923697-2.187231930871102j)
R: (0.7967230254805728+2.1372830998323336j) R_prev: (0.09741929659485182+2.278871674493936j)
R: (0.7967230254805728-2.1372830998323336j) R_prev: (0.09741929659485182-2.278871674493936j)
R: (-0.0643500086512605+2.280045110199885j) R_prev: (0.7967230254805728+2.1372830998323336j)
R: (-0.0643500086512605-2.280045110199885j) R_prev: (0.7967230254805728-2.1372830998323336j)
R: (-0.7673624424946057+2.147999420392972j) R_prev: (-0.0643500086512605+2.280045110199885j)
R: (-0.7673624424946057-2.147999420392972j) R_prev: (-0.0643500086512605-2.280045110199885j)
R: (-0.05301076359905974+2.280336924031726j) R_prev: (-0.767362442

R: (1.871079143353824+8.477600855833227j) R_prev: (1.450940510598776+8.559522449552722j)
R: (1.871079143353824-8.477600855833227j) R_prev: (1.450940510598776-8.559522449552722j)
R: (-2.386093682856915+8.347287602094918j) R_prev: (1.871079143353824+8.477600855833227j)
R: (-2.386093682856915-8.347287602094918j) R_prev: (1.871079143353824-8.477600855833227j)
R: (-0.8390626880042433+8.640985302769508j) R_prev: (-2.386093682856915+8.347287602094918j)
R: (-0.8390626880042433-8.640985302769508j) R_prev: (-2.386093682856915-8.347287602094918j)
R: (1.7728369341739283+8.498688277246806j) R_prev: (-0.8390626880042433+8.640985302769508j)
R: (1.7728369341739283-8.498688277246806j) R_prev: (-0.8390626880042433-8.640985302769508j)
R: (1.3099484976329894+8.582230969018271j) R_prev: (1.7728369341739283+8.498688277246806j)
R: (1.3099484976329894-8.582230969018271j) R_prev: (1.7728369341739283-8.498688277246806j)
R: (-2.4408171975130006+8.331450345329117j) R_prev: (1.3099484976329894+8.582230969018271j)


array([ 6.34280359+0.j        , -1.61418961+8.53024298j,
       -1.61418961-8.53024298j])